In [1]:
import numpy as np
from torchvision import datasets, models, transforms
from torchvision.models import ResNet18_Weights
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import copy
import matplotlib.pyplot as plt
import torchvision
from PIL import Image
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import tqdm
import pandas as pd

In [2]:
class HydraNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        for param in self.net.parameters():
            param.requires_grad = False

        self.n_features = self.net.fc.in_features
        self.net.fc = nn.Identity()

        # head for circle
        self.net.fc1 = nn.Sequential(nn.Linear(self.n_features, 2))

        # head for h_bar
        self.net.fc2 = nn.Sequential(nn.Linear(self.n_features, 2))

        # head for triangle
        self.net.fc3 = nn.Sequential(nn.Linear(self.n_features, 2))

        # head for v_bar
        self.net.fc4 = nn.Sequential(nn.Linear(self.n_features, 2))

    def forward(self, x):
        circle_head = self.net.fc1(self.net(x))
        h_bar_head = self.net.fc2(self.net(x))
        triangle_head = self.net.fc3(self.net(x))
        v_bar_head = self.net.fc4(self.net(x))
        return circle_head, h_bar_head, triangle_head, v_bar_head

In [3]:
def predict_image(im_name, model, device):
    transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
    img = Image.open(im_name)
    input = transform(img)
    input = input.to(device)
    input = input.unsqueeze(0)
    model.eval()
    circle_output, h_bar_output, triangle_output, v_bar_output = model(input)
    _, circle_preds = torch.max(circle_output, 1)
    _, h_bar_preds = torch.max(h_bar_output, 1)
    _, triangle_preds = torch.max(triangle_output, 1)
    _, v_bar_preds = torch.max(v_bar_output, 1)
    return {'im_name': im_name, 'circle_hn': circle_preds.item(), 'h_bar_hn': h_bar_preds.item(), 
            'triangle_hn': triangle_preds.item(), 'v_bar_hn': v_bar_preds.item()}


def generate_predictions_for_dataset(data_dir, model, device):
    """
    Function that takes in model, data directory containing two folders labeled with class names as input and generates
    predictions for all images in that directory
    :param data_dir:
    :param model:
    :param device:
    :param class_names:
    :return:
    """
    df_predictions = []
    for label in ['0', '1']:
        for im in os.listdir(data_dir + "/" + label):
            if im == ".DS_Store":
                continue
            full_im_path = data_dir + "/" + label + "/" + im
            prediction = predict_image(full_im_path, model, device)
            prediction['y'] = label
            df_predictions.append(prediction)
    return pd.DataFrame(df_predictions)

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = HydraNet().to(device=device)
model.load_state_dict(torch.load("HydraNet.pth"))

<All keys matched successfully>

In [5]:
df_predictions = generate_predictions_for_dataset("./data_full_pipeline", model, device)

In [9]:
df_predictions
df_predictions.to_csv("HydraNet_predictions_data_full_pipeline.csv")

In [10]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, penalty = None).fit(df_predictions[['circle_hn', 'h_bar_hn', 'triangle_hn', 'v_bar_hn']], 
                                                             df_predictions['y'])

In [12]:
clf.intercept_

array([-1.82530524])

In [13]:
df_predictions['img_name'] = df_predictions['im_name'].map(lambda x: x.split("/")[-1])
df_predictions

,im_name,circle_hn,h_bar_hn,triangle_hn,v_bar_hn,y,img_name
0,./data_full_pipeline/0/img_14654.png,0,0,0,0,0,img_14654.png
1,./data_full_pipeline/0/img_18633.png,0,0,0,1,0,img_18633.png
2,./data_full_pipeline/0/img_19073.png,1,0,1,0,0,img_19073.png
3,./data_full_pipeline/0/img_7048.png,0,0,0,0,0,img_7048.png
4,./data_full_pipeline/0/img_2716.png,1,0,0,0,0,img_2716.png
...,...,...,...,...,...,...,...
19995,./data_full_pipeline/1/img_2715.png,1,0,0,0,1,img_2715.png
19996,./data_full_pipeline/1/img_3155.png,1,1,1,1,1,img_3155.png
19997,./data_full_pipeline/1/img_13545.png,0,0,0,0,1,img_13545.png
19998,./data_full_pipeline/1/img_9779.png,0,1,1,1,1,img_9779.png


In [14]:
df_predictions.drop('im_name', axis=1, inplace=True)

In [15]:
df_predictions['y_hat'] = clf.predict(df_predictions[['circle_hn', 'h_bar_hn', 'triangle_hn', 'v_bar_hn']])
df_predictions = df_predictions.astype({'y': 'int64', 'y_hat': 'int64'})
hand_crafted_ft = pd.read_csv("hand_crafted_ft.txt", delimiter = "\t")
hand_crafted_ft
df_merged = pd.merge(hand_crafted_ft, df_predictions, on = ['img_name', 'y'])

In [16]:
df_merged

,img_name,h_bar,v_bar,circle,triangle,y,circle_hn,h_bar_hn,triangle_hn,v_bar_hn,y_hat
0,img_0.png,1,1,0,1,1,0,1,1,1,1
1,img_1.png,0,0,0,0,0,0,0,0,0,0
2,img_2.png,0,0,0,0,0,0,0,0,0,0
3,img_3.png,0,0,1,0,0,1,0,0,0,0
4,img_4.png,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
19995,img_19995.png,0,0,1,0,0,1,0,0,0,0
19996,img_19996.png,1,1,0,0,1,0,1,0,1,1
19997,img_19997.png,0,0,0,0,1,0,0,0,0,0
19998,img_19998.png,0,0,0,1,1,0,0,1,0,0


In [18]:
np.sum(df_merged["y"] == df_merged["y_hat"])/20000

0.7789

In [19]:
df_merged.to_csv("black_box_predictions_data_full_pipeline.csv", index = False)